In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    AdamW,
    AutoConfig
)
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed()

class CFG:
    max_len = 128
    batch_size = 16
    learning_rate = 4e-6
    epochs = 24
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    min_samples_per_class = 2  # 클래스당 최소 샘플 수

# 데이터 로드 및 전처리
df = pd.read_csv("predicted_yongin_departments.csv")

df.dropna(inplace=True)
df.drop_duplicates(subset=['title', 'complaint'], keep='first', inplace=True)

def normalize_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.strip()

df['title'] = df['title'].apply(normalize_text)
df['complaint'] = df['complaint'].apply(normalize_text)
df['text'] = df['title'] + ' [SEP] ' + df['complaint']

# 클래스별 샘플 수 확인
class_counts = df['yongin_department'].value_counts()
print("클래스별 샘플 수:")
print(class_counts)

# 최소 샘플 수 이상인 클래스만 선택
valid_classes = class_counts[class_counts >= CFG.min_samples_per_class].index
df_filtered = df[df['yongin_department'].isin(valid_classes)].copy()

print(f"\n필터링 전 데이터 수: {len(df)}")
print(f"필터링 후 데이터 수: {len(df_filtered)}")
print(f"제외된 클래스 수: {len(class_counts) - len(valid_classes)}")

# 레이블 인코딩
label_encoder = {label: i for i, label in enumerate(df_filtered['yongin_department'].unique())}
df_filtered['label'] = df_filtered['yongin_department'].map(label_encoder)

# 학습 및 검증 데이터 분할
train_df, val_df = train_test_split(
    df_filtered, 
    test_size=0.2, 
    stratify=df_filtered['yongin_department'], 
    random_state=42
)

/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.20) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


클래스별 샘플 수:
yongin_department
위생과         37
건축과         36
도로관리과       32
공동주택과       30
주택정책과       25
보건행정과       23
환경정책과       21
토지정보과       21
교통정책과       19
보건정책과       19
주택정비과       18
대중교통과       15
건설정책과       11
도시정책과       11
공원조성과       10
동부공원관리과     10
자원순환과        9
도서관정책과       8
자원육성과        7
도시정비과        5
공공건축과        5
도시개발과        5
기후대기과        5
생태하천과        4
건강증진과        4
하수관로관리과      4
도로구조물과       4
반도체일반산단과     2
하수시설과        2
물류화물과        2
반도체정책과       2
미래성장전략과      2
기술지원과        2
농촌테마과        1
하수행정과        1
하수도사업소       1
미래도시과        1
기업산단입지과      1
반도체국가산단과     1
Name: count, dtype: int64

필터링 전 데이터 수: 416
필터링 후 데이터 수: 410
제외된 클래스 수: 6


In [2]:
# 토크나이저 로드
model_path = "klue-roberta-large" 
tokenizer = AutoTokenizer.from_pretrained(model_path) 

# 설정 로드 및 수정
config = AutoConfig.from_pretrained(model_path)
config.num_labels = len(label_encoder)  

# 모델 생성
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    config=config
)

# 저장된 가중치 로드
state_dict = torch.load(f"{model_path}/pytorch_model.bin")

# 불필요한 키 제거
for key in list(state_dict.keys()):
    if key.startswith('lm_head') or key == 'roberta.embeddings.position_ids':
        del state_dict[key]

# 모델에 가중치 로드
model.load_state_dict(state_dict, strict=False)

# GPU로 모델 이동 (필요한 경우)
model.to(CFG.device)

print("Model loaded successfully!")


class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item] if self.labels is not None else -1
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }
# 데이터셋 및 DataLoader 생성
train_dataset = TextDataset(train_df['text'].values, train_df['label'].values, tokenizer)
val_dataset = TextDataset(val_df['text'].values, val_df['label'].values, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG.batch_size)

# Optimizer 설정
optimizer = AdamW(model.parameters(), lr=CFG.learning_rate)

# 학습 및 검증
for epoch in range(CFG.epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{CFG.epochs}'):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(CFG.device)
        attention_mask = batch['attention_mask'].to(CFG.device)
        labels = batch['labels'].to(CFG.device)
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch + 1}/{CFG.epochs}, Train Loss: {train_loss / len(train_loader):.4f}")
    
    # 검증
    model.eval()
    val_predictions = []
    val_true_labels = []
    val_loss = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc='Validating'):
            input_ids = batch['input_ids'].to(CFG.device)
            attention_mask = batch['attention_mask'].to(CFG.device)
            labels = batch['labels'].to(CFG.device)
            
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            
            _, preds = torch.max(outputs.logits, dim=1)
            val_predictions.extend(preds.cpu().tolist())
            val_true_labels.extend(labels.cpu().tolist())
    
    val_f1 = f1_score(val_true_labels, val_predictions, average='macro')
    val_accuracy = accuracy_score(val_true_labels, val_predictions)
    print(f"Validation Loss: {val_loss / len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")

# 모델 저장
torch.save(model.state_dict(), 'yongin_department_classifier.pt')
print("Model saved successfully!")

# 레이블 인코더 저장
import json
with open('label_encoder.json', 'w') as f:
    json.dump(label_encoder, f)
print("Label encoder saved successfully!")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_97744/3153571581.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicit

Model loaded successfully!


Epoch 1/24: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:07<00:00,  3.00it/s]


Epoch 1/24, Train Loss: 3.4934


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.98it/s]


...

Validation Loss: 1.2315, Validation Accuracy: 0.6707, Validation F1 Score: 0.5330


Epoch 24/24: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:05<00:00,  3.73it/s]


Epoch 24/24, Train Loss: 0.2486


Validating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.87it/s]


Validation Loss: 1.2533, Validation Accuracy: 0.6707, Validation F1 Score: 0.5306
Model saved successfully!
Label encoder saved successfully!


In [4]:


def predict_department(title, complaint):
    # 텍스트 전처리
    text = f"{title} [SEP] {complaint}"
    
    # 토크나이징
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    # 모델 입력 준비
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs.logits, dim=1)
    
    # 예측 결과 디코딩
    predicted_label = label_decoder[preds.item()]
    
    # 확률값 계산
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
    confidence = probabilities[0][preds].item()
    
    return predicted_label, confidence

# 예시 사용
title = input("민원 제목을 입력하세요: ")
complaint = input("민원 내용을 입력하세요: ")

department, confidence = predict_department(title, complaint)
print("\n=== 예측 결과 ===")
print(f"배정 부서: {department}")
print(f"신뢰도: {confidence*100:.2f}%")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_97744/2169166688.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicit

Model loaded successfully!


민원 제목을 입력하세요:  물의 수질에 대한 불만
민원 내용을 입력하세요:  제발 깨끗한 물을 먹고 싶어요



=== 예측 결과 ===
배정 부서: 환경정책과
신뢰도: 64.88%
